<a href="https://colab.research.google.com/github/Antonioarf/blox/blob/master/recomenda%C3%A7%C3%A3o/Recommender-t1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FECAF 1

In [0]:
import heapq
import pandas as pd;
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [0]:
df1 = pd.read_csv("https://raw.githubusercontent.com/Antonioarf/blox/master/recomenda%C3%A7%C3%A3o/Relat%C3%B3rio%20sem%20t%C3%ADtulo_P%C3%A1gina%201_Tabela.csv")
df2 = pd.read_csv("https://raw.githubusercontent.com/Antonioarf/blox/master/recomenda%C3%A7%C3%A3o/Relat%C3%B3rio%20sem%20t%C3%ADtulo_Tabela.csv") 
# df1.head(3)
# df2.head(3)
# df_t.head(3)
# df3.head(3)
# df4.head(3)
# dfx.head(3)

## limpesa e dummify do df1
### estamos trabalhando com 113 blox e 678 alunos

In [0]:
df1.head()
dfx = df1
dfx = dfx.set_index('nome_do_aluno')
dfx = dfx.sort_values("nome_do_aluno")
entao = dfx['nome_blox']

In [0]:
def dummify(data, column_name):
    """
        Converte a coluna column_name em dummies / one-hot e as adiciona ao dataframe
        retorna uma copia do  df original *sem* a coluna que foi dummified
    """
    df = data.copy()
    # Nota: se você for um econometrista, pode se interessar por usar a opção dropFirst = True na get_dummies
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=None)], axis=1)
    return df2

def deletar(coluna):
    lista = list (coluna.unique()) #.tolinst()  
    for e in lista:
        coluna = coluna.replace(e,lista.index(e))
    return coluna

In [0]:
df1 = dummify(df1, ['nome_blox'])
lista = df1.columns.tolist()[5:]

In [0]:
aggregation_functions = {}
aggregation_functions['course_name'] = 'first'
for e in lista:
    aggregation_functions[e] = 'sum'
df1 = df1.groupby(df1['nome_do_aluno']).aggregate(aggregation_functions)

In [0]:
lista = ["course_name"]
for e in df1.columns[1:]:
    lista.append(e[10:])
df1.columns = lista

## Limpesa e tratamento do df2

## Primeira tentativa de unspervised learning do df1

In [0]:
df_t = pd.DataFrame()
for e in df1.columns[1:]:
    lista = list (df2.loc[df2['nome_blox'] == e ]["area_funcional"])
    df_t[e] = df1[e].replace(1, lista[0] )#.replace(0,np.nan)

In [0]:
df3 = df1
df3 = df3.drop(list(df1.columns),axis=1)
lista = []
for e in df_t.values.tolist():
    lista.append(list(filter(lambda a: a != 0, e)))
df3['lista'] = lista

In [0]:
lll = []
for i in df3['lista']:
    k = i[0]
    lista = []
    for e in df2["area_funcional"].unique():
        lista.append(e.count(k))
    lll.append(lista)
df4 = pd.DataFrame.from_records(lll,columns = df2["area_funcional"].unique())

In [0]:
df4['corse_name'] = df1['course_name'].tolist()
df4['corse_name'] = df4["corse_name"].astype('category')
lista=[]
for e in dfx.index.unique():
    lista.append(dfx.loc[dfx.index == e]["nome_blox"].tolist())
df4["resp"] = lista

In [13]:
df4.index = df3.index
# dfx = dfx.set_index('nome_do_aluno')
dfx = dfx.drop(["nome_blox","course_name","modalidade","ch","nota"],axis = 1)
dfx.cloumns = df4.columns
dfx.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.


(5827, 0)

In [0]:
dfx = dfx.merge(df4, left_on= 'nome_do_aluno', right_on= 'nome_do_aluno')

In [0]:
dfx = dummify(dfx, 'corse_name')
dfx['resp']= entao.astype('category')

In [0]:
X_train, X_test, y_train, y_test = train_test_split((dfx.drop(["resp"],axis =1)) ,dfx["resp"], test_size=0.05,random_state=100)

In [0]:
lis = y_train.unique()
def sugestao(X_train,y_train,X_test):
    rdm = SVC(kernel='linear', random_state=112)
    rdm = rdm.fit(X_train,y_train)
    y_pred = rdm.predict(X_test)
    j = list (rdm.decision_function(X_test))
    sugestoes = []
    t = 0
    for e in j:
        sugestao = []
        while len(sugestao)< 3:
            x = 3
            maiores_index = heapq.nlargest(x, range(len(e)), e.take)
            listinha = []
            for m in maiores_index:
                e = list(e)
                listinha.append(lis[m])
                sugestao.append(list(set(listinha) - set(df3["lista"][t])))
                
            x+=1
        sugestoes.append(sugestao[-1])
        t+=1
    return sugestoes

deu = sugestao(X_train,y_train,X_test)